# Materialized Views in Oracle

### Base Table Setup
We first create the base table `DEP` and insert sample department records.

In [ ]:
CREATE TABLE dep (
    deptno   NUMBER(2)       PRIMARY KEY,
    dname    VARCHAR2(14)    NOT NULL,
    loc      VARCHAR2(13)
);

INSERT INTO dep VALUES (10,'ACCOUNTING','NEW YORK');
INSERT INTO dep VALUES (20,'RESEARCH','DALLAS');
INSERT INTO dep VALUES (30,'SALES','CHICAGO');
INSERT INTO dep VALUES (40,'OPERATIONS','BOSTON');
COMMIT;

In [ ]:
SELECT * FROM dep;

### Creating a Materialized View
A materialized view stores the results of a query physically on disk.

In [ ]:
CREATE MATERIALIZED VIEW m1 AS SELECT * FROM dep;

In [ ]:
SELECT * FROM m1;

In [ ]:
SELECT mview_name FROM user_mviews WHERE mview_name = 'M1';

### DML Restriction
You cannot perform DML directly on a materialized view. Any changes must be made to the base table.

In [ ]:
INSERT INTO m1 VALUES (50,'ENGINEERING','BENGALURU'); -- ORA-01732

### Updating Base Table
Insert into the base table `DEP`. The materialized view will not auto-refresh unless specified.

In [ ]:
INSERT INTO dep VALUES (50,'ENGINEERING','BENGALURU');

In [ ]:
SELECT * FROM dep;

In [ ]:
SELECT * FROM m1; -- Still old data until refreshed

### Manual Refresh
Use `DBMS_MVIEW.REFRESH` to update the materialized view with the latest data.

In [ ]:
EXEC DBMS_MVIEW.REFRESH('M1', 'C');

In [ ]:
SELECT * FROM m1;

### Auto Refresh on Commit
Create a materialized view that automatically refreshes whenever a transaction on the base table is committed.

In [ ]:
CREATE MATERIALIZED VIEW m2
REFRESH ON COMMIT
AS SELECT * FROM dep;

In [ ]:
INSERT INTO dep VALUES (60,'DEVELOPMENT','BENGALURU'); COMMIT;

In [ ]:
SELECT * FROM m2; -- Auto refreshed after commit

### Effect of Dropping Base Table
When the base table is dropped, the materialized views are not automatically dropped. They continue to exist but may not reflect correct results or refresh properly.

In [ ]:
DROP TABLE dep;

In [ ]:
SELECT * FROM m1; -- Still returns old data, but cannot be refreshed

In [ ]:
SELECT * FROM m2; -- Still returns old data, but cannot be refreshed